# A Flixible CNN
Layers determind by [link](https://iq.opengenus.org/output-size-of-convolution/)

In [2]:
import torch
import torchvision
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch import optim

Download data

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Plotting some of the data

In [ ]:
# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

## Define NN
Define the class!

In [81]:
channel_data = testset[0][0].shape[0] # num channels 
data_size = testset[0][0].shape[1]
feature1 = [6,2,2]  # NOTE feature num, stide, padding
feature2 = [16,2,2] # NOTE feature num, stide, padding
out_size = len(classes)
k_conv_size = 5 # 5x5 convolution kernel
k_pool = [2,2] # pool layer 1, layer 2


def get_output_shape(I, K, P, S, M):
    '''
    compute the output shape of a convolutional layer
    for the computation, you can refer to 
    https://androidkt.com/calculate-output-size-convolutional-pooling-layers-cnn/
    https://stackoverflow.com/questions/53580088/calculate-the-output-size-in-convolution-layer

    I: image size
    K: kernel size
    P: padding size
    S: stride
    M: pooling size
    '''
    return ((I-K+2*P)//S+1)//M

Conv_out1 = get_output_shape(data_size,k_conv_size,feature1[2],feature1[1],k_pool[0])
Conv_out2 = get_output_shape(Conv_out1,k_conv_size,feature2[2],feature2[1],k_pool[1])

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels= channel_data,out_channels= feature1[0], kernel_size= k_conv_size, stride= feature1[1], padding= feature1[2]), 
            nn.BatchNorm2d(feature1[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=k_pool[0])
            )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels= feature1[0],out_channels= feature2[0], kernel_size= k_conv_size, stride= feature2[1], padding= feature2[2]), 
            nn.BatchNorm2d(feature2[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=k_pool[1])
            )
        
        
        self.fc = nn.Linear(feature2[0] * int(Conv_out2)**2, out_size)
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        
        return out



### Define a Loss function and optimizer

In [82]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.001)

Training

In [83]:
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.906
[1,  4000] loss: 1.683
[1,  6000] loss: 1.601
[1,  8000] loss: 1.553
[1, 10000] loss: 1.539
[1, 12000] loss: 1.484
[2,  2000] loss: 1.461
[2,  4000] loss: 1.443
[2,  6000] loss: 1.405
[2,  8000] loss: 1.415
[2, 10000] loss: 1.424
[2, 12000] loss: 1.395
[3,  2000] loss: 1.362
[3,  4000] loss: 1.377
[3,  6000] loss: 1.361
[3,  8000] loss: 1.352
[3, 10000] loss: 1.361
[3, 12000] loss: 1.348
[4,  2000] loss: 1.338
[4,  4000] loss: 1.347
[4,  6000] loss: 1.333
[4,  8000] loss: 1.321
[4, 10000] loss: 1.303
[4, 12000] loss: 1.329
[5,  2000] loss: 1.294
[5,  4000] loss: 1.299
[5,  6000] loss: 1.311
[5,  8000] loss: 1.314
[5, 10000] loss: 1.285
[5, 12000] loss: 1.294
[6,  2000] loss: 1.291
[6,  4000] loss: 1.268
[6,  6000] loss: 1.287
[6,  8000] loss: 1.277
[6, 10000] loss: 1.304
[6, 12000] loss: 1.284
[7,  2000] loss: 1.267
[7,  4000] loss: 1.267
[7,  6000] loss: 1.266
[7,  8000] loss: 1.289
[7, 10000] loss: 1.282
[7, 12000] loss: 1.276
[8,  2000] loss: 1.260
[8,  4000] 

## Test